In [ ]:
import cv2
import matplotlib.image as img
import matplotlib.pyplot as plt
import numpy as np
import random
import os
from scipy.stats import wasserstein_distance
from imageio import imread
from PIL import ImageFile
import pandas as pd

ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
def get_histogram(img):
    h, w = img.shape
    hist = [0.0] * 256
    for i in range(h):
        for j in range(w):
            hist[img[i, j]] += 1
    return np.array(hist) / (h * w) # 히스토그램 

In [ ]:
sift = cv2.xfeatures2d.SIFT_create()
bfMatcher = cv2.BFMatcher()

In [ ]:
df1 = pd.read_csv(r'C:\Users\User\Desktop\동일이미지_하은겸\담당별데이터\강원인프라담당_루즈튜브.csv', index_col=0)

In [ ]:
df1.head()

,업체명,담당,작업후 사진_루즈튜브_위경도,작업후 사진_루즈튜브_첨부일시,작업후 사진_루즈튜브_파일경로(서버의 디렉토리+파일명),루즈튜브,month,day,사진파일명
0,(주)신라이앤씨,강원인프라담당,"129.0508709, 37.2292454",2020/01/02 10:36:13,/toss_nas/upload/1/srch/wk/2020/1/2/e56460e127...,WKS-191231-00551-0001_202001021020_53315383_루즈튜브,1,2,e56460e1278147200102_6_687c4c6b8a024bd7a99013f...
1,화인텔레콤(주),강원인프라담당,"129.0501675, 37.2299069",2020/01/02 10:43:55,/toss_nas/upload/1/srch/wk/2020/1/2/cfc79ea76f...,WKS-191231-00551-0001_202001021007_52439995_루즈튜브,1,2,cfc79ea76f3e4b200102_6_f873e337cd964aa790fc317...
2,(주)신라이앤씨,강원인프라담당,"129.0498737, 37.2296217",2020/01/02 10:54:50,/toss_nas/upload/1/srch/wk/2020/1/2/d1c30ed5a7...,WKS-191231-00551-0001_202001021045_53315389_루즈튜브,1,2,d1c30ed5a70a4b200102_6_64bb644853a542fdb7ce39b...
3,(합)동양아이텍,강원인프라담당,"127.2171442, 38.2071482",2020/01/02 10:14:19,/toss_nas/upload/1/srch/wk/2020/1/2/124681e8e1...,WKS-191230-01053-0001_202001020947_53034251_루즈튜브,1,2,124681e8e16644200102_6_4423c5a7e1354394ab96f7c...
4,(합)동양아이텍,강원인프라담당,"127.2125969, 38.2067648",2020/01/02 11:40:18,/toss_nas/upload/1/srch/wk/2020/1/2/45e22c4d63...,WKS-191230-01167-0001_202001021128_135316750_루즈튜브,1,2,45e22c4d63db47200102_6_e54d9014b51d4152ab94d18...


In [ ]:
df1.shape

(42633, 9)

In [ ]:
two_month = list(range(2,31))
two_month

[2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30]

In [ ]:
make_list = []

In [ ]:
df1_sample = df1[df1['month'].isin([9])] # 9월
df1_sample.head()

,업체명,담당,작업후 사진_루즈튜브_위경도,작업후 사진_루즈튜브_첨부일시,작업후 사진_루즈튜브_파일경로(서버의 디렉토리+파일명),루즈튜브,month,day,사진파일명
24468,주식회사 한솔텔레콤,강원인프라담당,"128.6072257, 38.0564229",2020/09/01 05:19:26,/toss_nas/upload/1/srch/wk/2020/9/1/6ced7bc703...,WKS-200831-01575-0001_202009010442_114953_루즈튜브,9,1,6ced7bc703184b200901_6_774a3d5a0a1e41aab0c7329...
24469,주식회사 한솔텔레콤,강원인프라담당,"128.6173308, 38.07354",2020/09/01 01:31:25,/toss_nas/upload/1/srch/wk/2020/9/1/7a5a16e435...,WKS-200831-01575-0001_202009010124_52933240_루즈튜브,9,1,7a5a16e435f44c200901_6_080b2a2418e54cab9543401...
24470,주식회사 한솔텔레콤,강원인프라담당,"128.6129251, 38.0738618",2020/09/01 01:49:13,/toss_nas/upload/1/srch/wk/2020/9/1/bfd4d26b3d...,WKS-200831-01575-0001_202009010129_52933241_루즈튜브,9,1,bfd4d26b3d6b41200901_6_2253047c9feb48138768f39...
24471,주식회사 한솔텔레콤,강원인프라담당,"128.6120729, 38.0737471",2020/09/01 01:49:57,/toss_nas/upload/1/srch/wk/2020/9/1/2f6a3c384d...,WKS-200831-01575-0001_202009010138_52957137_루즈튜브,9,1,2f6a3c384d454a200901_6_71652113efa947a08d83ea5...
24472,주식회사 한솔텔레콤,강원인프라담당,"128.6063808, 38.0732838",2020/09/01 02:14:15,/toss_nas/upload/1/srch/wk/2020/9/1/15236c2cc2...,WKS-200831-01575-0001_202009010154_52577185_루즈튜브,9,1,15236c2cc20546200901_6_8384ef5ef8b3437887236bf...


In [ ]:
df1_sample.shape

(4489, 9)

In [ ]:
pic_month = list(df1_sample['month'])
pic_day= list(df1_sample['day'])
pic_name = list(df1_sample['사진파일명'])

In [ ]:
file_name = []

In [ ]:
for i in range(len(df1_sample)):
    pic =str(pic_month[i])+ '/' + str(pic_day[i]) +'/'+pic_name[i]
    file_name.append(pic)

In [ ]:
file_name

['9/1/6ced7bc703184b200901_6_774a3d5a0a1e41aab0c7329ad96025cb.jpg',
 '9/1/7a5a16e435f44c200901_6_080b2a2418e54cab9543401ed154b8b5.jpg',
 '9/1/bfd4d26b3d6b41200901_6_2253047c9feb48138768f39243868eb4.jpg',
 '9/1/2f6a3c384d454a200901_6_71652113efa947a08d83ea59e25eeff8.jpg',
 '9/1/15236c2cc20546200901_6_8384ef5ef8b3437887236bf1daecbcf1.jpg',
 '9/1/fea175c6237a47200901_6_7f739140576444eebd26f7b2a5435266.jpg',
 '9/1/b085f43e136a4a200901_6_bfcd1b8eb57649e9af90abe634d17fc6.jpg',
 '9/1/de39dfe924224b200901_6_332b904828cf41f2b729e3b24759f2ec.jpg',
 '9/1/bebe33d8b75c49200901_6_805f99787c4e408b8f95747c78f1f6e8.jpg',
 '9/1/f4ba26ca955e49200901_6_36fdc8a572a7466e9eb520f1d3082072.jpg',
 '9/1/e830124bd73244200901_6_0fa60e03e06f4a2cb9690924134d90bc.jpg',
 '9/1/5fc3c827156843200901_6_03fc49a6fd8d459e8635a5fed4cfa806.jpg',
 '9/1/9fd9c0a526ff48200901_6_00a65de7a8494e2899266338b58a9c28.jpg',
 '9/1/bd711f2f244e49200901_6_52a69e84f9ac4b2ba08544584e3ecaa7.jpg',
 '9/1/115b4428b3eb4f200901_6_20204a04506648aa819

In [ ]:
temp_df5 = pd.DataFrame(index=range(0,0), columns=['name','keyPointDescriptor','array'])

In [ ]:
temp_df5

,name,keyPointDescriptor,array


In [ ]:
index = 0
for i in file_name:
    file_name = "D:/" + i
    #print(pic_name[j])
    try:
        rotateImageNDArray = cv2.imread(file_name)
        rotateGrayscaleImagenDArray = cv2.cvtColor(rotateImageNDArray, cv2.COLOR_BGR2GRAY)
        rotateKeyPointList, rotateKeyPointDescriptor = sift.detectAndCompute(rotateGrayscaleImagenDArray, None)
        input_image2 = imread(file_name, pilmode='L')
        input_image_hist = get_histogram(input_image2)
        temp_list = {'name':pic_name[index], 'keyPointDescriptor':[rotateKeyPointDescriptor],'array':[input_image_hist]}
        temp_df5 = temp_df5.append(temp_list, ignore_index=True)
    except:
        temp_list = {'name':pic_name[index], 'keyPointDescriptor':None,'array':None}
        temp_df5 = temp_df5.append(temp_list, ignore_index=True)
        
    index = index+1
    print(index)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [ ]:
temp_df5

In [ ]:
temp_df5.to_csv('9_강원인프라담당_루즈튜브.csv')